In [31]:
from __future__ import print_function, division
from future.utils import iteritems
from builtins import range, input
import numpy as np
from datetime import datetime
from scipy.stats import norm
import pandas as pd

from scipy.stats import multivariate_normal as mvn

class NaiveBayes(object):
    def fit(self, X, Y, smoothing=1e-2): #smoothing sıfıra bölme hatasını engellemek için , X : 1x784 ( BW) boyutlu 0-1 Y: rakam karşılığı
        self.gaussians = dict() # null olmasın diye başlangıçta oluşturduk
        self.priors = dict()
        labels = set(Y) 
        for c in labels:
            current_x = X[Y == c]
            self.gaussians[c] = {
                'mean': current_x.mean(axis=0),
                'var': current_x.var(axis=0) + smoothing,
            }
            self.priors[c] = float(len(Y[Y == c])) / len(Y)
    
    def score(self, X, Y):
        P = self.predict(X)
        return np.mean(P == Y)

    def predict(self, X):
        N, D = X.shape
        K = len(self.gaussians)
        P = np.zeros((N, K))
        for c, g in iteritems(self.gaussians):
            mean, var = g['mean'], g['var']
            P[:,c] = mvn.logpdf(X, mean=mean, cov=var) + np.log(self.priors[c])
        return np.argmax(P, axis=1)
def get_data(limit=None):
    print("Reading in and transforming data...")
    df = pd.read_csv('train.csv')
    data = df.values
    np.random.shuffle(data)
    X = data[:, 1:] / 255.0 # data is from 0..255
    Y = data[:, 0]
    if limit is not None:
        X, Y = X[:limit], Y[:limit]
    return X, Y

X : de 5000 tane veri var her birinin boyutu 1x786,
Y : de 5000 tane veri var her birinin boyutu 1x1

set(Y) # her bir veriyi ağaç yapısı olarak alıyormuş

# set(Y) 'nin karmaşıklığı nedir ?

5000 taneyi dolaş bunlardan bir tane set labels oluştur . Dolayısıyla karmaşıklık 5000 dir.
Örnek :
liste var L1 = [1,2,1,4,6,1,2,3,0]
labels = [1,2,4,6,3,0]
şeklinde oluyor.Ama hepsini dolaşmak zorunda olduğu için karmaşıklık O(n)

# for c in labels : karmaşıklığı : O(|labels|)

    ## current_x = X[Y == c] 
başlangıçta c = 0 Y değeri 5000 tane y nin 0 olduğu değerleri alıp x'e yaz diyor
yani y si c ile uyumlu olan x leri al diyor.
1 2 1 0 2   0
1 0 3 2 0   0
3 2 0 1 3   1
_________   __
      x      y
      
c_x = X[y==0]
        => 1 2 1 0 2
           1 0 3 2 0
           
# Yani for c in label : karmaşıklık 

    #### current_x = X[Y == c] -> O(n.m)
    #### self.gaussians[c]   -> O(s.m)  s: of instance of '7' m:786
   
# self.priors[c] = float(len(Y[Y == c])) / len(Y)
    
    7 den kaç tane var , yüzdelik olarak hesaplıyor. Karmaşıklık Y sayısı yani n'dir. O(n)

# Predict(self, x) : Karmaşıklığı

### for c, g in iteritems(self.gaussians):
    
    * gaussian sayısı 10 
    # mean, var = g['mean'], g['var']
      P[:,c] = mvn.logpdf(X, mean=mean, cov=var) + np.log(self.priors[c])
      
      Yukarıdaki kodlarda bir formül olduğu için kısaca O(1) diyebiliriz.
      
      O halde for döngüsünün karmaşıklığı 10Dur.
### return np.argmax(P, axis=1)
       
       bu kodda da yine 10 boyutlu olduğu için karmaşıklık 10'dur
        

In [32]:
X, Y = get_data(10000)
Ntrain = len(Y) // 2
Xtrain, Ytrain = X[:Ntrain], Y[:Ntrain]
Xtest, Ytest = X[Ntrain:], Y[Ntrain:]

model = NaiveBayes()
t0 = datetime.now()
model.fit(Xtrain, Ytrain)
print("Training time:", (datetime.now() - t0))

t0 = datetime.now()
print("Train accuracy:", model.score(Xtrain, Ytrain))
print("Time to compute train accuracy:", (datetime.now() - t0), "Train size:", len(Ytrain))

t0 = datetime.now()
print("Test accuracy:", model.score(Xtest, Ytest))
print("Time to compute test accuracy:", (datetime.now() - t0), "Test size:", len(Ytest))


Reading in and transforming data...
Training time: 0:00:00.124800
Train accuracy: 0.8188
Time to compute train accuracy: 0:00:07.032800 Train size: 5000
Test accuracy: 0.7958
Time to compute test accuracy: 0:00:05.555400 Test size: 5000
